In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

from scipy.signal import find_peaks
from collections import Counter
from carmodel_calibration.data_integration.helper import (
    get_lane, get_angle, get_difference)
from carmodel_calibration.helpers import _get_starting_time, _get_vehicle_meta_data
from carmodel_calibration.data_integration.data_set import DataSet
filename = DataSet(Path(os.environ["DATA_DIR"])).get_filename_by_id(1)
data, meta_data, lane_data = DataSet.read_file(Path(filename))
data.to_csv(".tmp/data.csv", index=False)
meta_data.to_csv(".tmp/meta_data.csv", index=False)
lane_data.to_csv(".tmp/lane_data.csv", index=False)

In [2]:
def car_goes_straight(lane_data, data_chunk):
    """calculate distance (int) between cars from dataframe chunks"""
    if "lane" not in data_chunk.columns:
        data_chunk["lane"] = get_lane(data_chunk["xCenter"],
                                      data_chunk["yCenter"],
                                      lane_data, True)
    lane = data_chunk["lane"].values
    # TODO: instead of counter, integrate the lane over the traveled distance
    # for more accurate lane identification
    lane_counts = sorted(list(Counter(lane).values()), reverse=True)
    lane_mean = np.rint(np.mean(lane))
    lane_xy = (lane_data[lane_data["trackId"]==lane_mean]
               [["xCenter", "yCenter"]].values[::10])
    # xy_center = data_chunk[["xCenter", "yCenter"]].values[:-100:10]
    xy_center = data_chunk[["xCenter", "yCenter"]].values[::10]
    distance =  get_difference(xy_center[:,0], xy_center[:,1],
                              lane_xy[:,0], lane_xy[:,1])
    if any(distance > 15):
        return False
    if np.mean(distance) > 7:
        return False
    if len(lane_counts) < 2:
        return True
    # return True
    return lane_counts[1] / lane_counts[0] < 0.1
def angle_diff(traj_a, traj_b):
    """both trajectories in degree"""
    diff = traj_a - traj_b
    return (diff + 180) % 360 - 180

def euclidian_distance(pos1, pos2):
    """calculate the euclidian distance between 2points in n-dim space"""
    return np.sum(np.square(pos1-pos2), axis=1) ** 0.5

In [3]:
def following_cars(data: pd.DataFrame, lane_data: pd.DataFrame,
                   meta_data: pd.DataFrame, use_xy=False):
    """
    generator that gets ids of cars that are classified as following each other
    :yield:         yields tuple of IDs of 2 cars , that are following each
                    other, and their distance as np array
    """
    dist_threshold = 100
    speed_threshold = 1.5
    min_gap_treshold = 0.1
    abscissa = "xCenter" if use_xy else "lon"
    ordinate = "yCenter" if use_xy else "lat"
    unique_tracks = data["trackId"].unique()
    crossing_times = np.zeros((int(
        unique_tracks.shape[0])))
    for track_id, track_name in enumerate(unique_tracks):
        crossing_time = data[data["trackId"]==track_name]["intersectionCrossing"]
        crossing_time = crossing_time.values[0]
        crossing_times[track_id] = crossing_time
    for leader_id, leader in enumerate(unique_tracks):
        data_chunk = data[data["trackId"]==leader]
        leader_meta, _ = _get_vehicle_meta_data(
            meta_data, (leader, None, meta_data.iloc[0]["recordingId"]))
        if leader_meta["class"] not in ["Car", "Van"]:
            continue
        if leader in [108, 112, 92]:
            print("debug")
        starting_time_l = _get_starting_time(data_chunk)
        if "lane" not in data_chunk.columns:
            data_chunk["lane"] = get_lane(data_chunk[abscissa],
                                          data_chunk[ordinate],
                                          lane_data)
        # check lane
        if not car_goes_straight(lane_data, data_chunk):
            continue
        if crossing_times[leader_id] == 0:
            continue
        if not any(data_chunk["speed"] < speed_threshold):
            continue
        if crossing_times[leader_id] == 0:
            continue
        # cars that are in the same timeframe, with the same heading
        # and position
        lane = np.rint(np.mean(data_chunk["lane"].values))
        # do not consider zero values in mean for heading
        heading = np.true_divide(data_chunk["heading"].values.sum(0),
                                 (data_chunk["heading"].values!=0).sum(0))
        leader_frames = data_chunk["frame"].values
        next_cars = np.argsort(
            np.abs(crossing_times - crossing_times[leader_id]))
        for follower_id in next_cars:
            follower = unique_tracks[follower_id]
            if follower == leader:
                continue
            next_chunk = data[data["trackId"] == follower]
            leader_meta, follower_meta = _get_vehicle_meta_data(
                meta_data, (leader, follower, 1))
            if len(next_chunk) == 0:
                continue
            if follower_meta["class"] not in ["Car", "Van"]:
                continue
            overlapping_frames = np.intersect1d(
                next_chunk["frame"].values, leader_frames)
            if overlapping_frames.shape[0] == 0:
                continue
            start = (
                next_chunk[next_chunk["frame"]==overlapping_frames[0]
                           ]["time"].values[0])
            stop = (
                next_chunk[next_chunk["frame"]==overlapping_frames[-1]
                           ]["time"].values[0])
            overlapping_time = stop - start
            # check that follower follows at least 5 seconds
            if overlapping_time < 5:
                continue
            if not car_goes_straight(lane_data, next_chunk):
                # print("because of not straight")
                continue
            # check lane
            if "lane" not in next_chunk.columns:
                next_chunk["lane"] = get_lane(next_chunk[abscissa],
                                              next_chunk[ordinate],
                                              lane_data)
            follower_lane = np.rint(np.mean(next_chunk["lane"].values))
            if follower_lane != lane:
                continue
            # check heading
            next_heading = np.true_divide(next_chunk["heading"].values.sum(0),
                                 (next_chunk["heading"].values!=0).sum(0))
            if angle_diff(heading, next_heading) > 20:
                continue
            # get position in meters
            overlap_leader = data_chunk[
            data_chunk["frame"].isin(overlapping_frames)]
            pos_leader = overlap_leader[["xCenter","yCenter"]].values
            overlap_follower = next_chunk[
                next_chunk["frame"].isin(overlapping_frames)]
            pos_follower = overlap_follower[["xCenter", "yCenter"]].values
            # check distance
            distance = euclidian_distance(pos_leader, pos_follower)
            if any(distance > dist_threshold):
                continue
            # get position in lat lon
            overlap_leader = data_chunk[
            data_chunk["frame"].isin(overlapping_frames)]
            pos_leader = overlap_leader[[abscissa,ordinate]].values
            overlap_follower = next_chunk[
                next_chunk["frame"].isin(overlapping_frames)]
            pos_follower = overlap_follower[[abscissa, ordinate]].values
            # check that follower is behind leader
            attack_vector = pos_leader - pos_follower
            attack_vector_angle = get_angle(attack_vector)
            condition = (
                (np.abs(angle_diff(attack_vector_angle, heading)) > 50)
                & (overlap_leader["speed"].values > 1)
                & (overlap_follower["speed"].values > 1)
            )
            if np.any(condition):
                continue
            time_ss, follower_distance_ss = (
                next_chunk.iloc[next_chunk["speed"].argmin()]
                [["time","distanceIntersectionCrossing"]])
            leader_distance_ss = (
                data_chunk.iloc[(data_chunk["time"] - time_ss).abs().argmin()]
                ["distanceIntersectionCrossing"])
            condition = (
                (data["lane"]==lane)
                & ((data["distanceIntersectionCrossing"]
                   < leader_distance_ss) & (np.isclose(data["time"],
                                                       time_ss,
                                                       atol=0.5)))
                & ((data["distanceIntersectionCrossing"]
                   > follower_distance_ss) & (np.isclose(data["time"],
                                                         time_ss,
                                                         atol=0.5)))
                & (~data["trackId"].isin([leader, follower_id]))
            )
            pos_follower = next_chunk[next_chunk["time"]==starting_time_l]
            pos_leader = data_chunk[data_chunk["time"]==starting_time_l]
            if len(pos_follower) > 0:
                pos_follower_xy = pos_follower[["xCenter", "yCenter"]].values[0]
                pos_leader_xy = pos_leader[["xCenter", "yCenter"]].values[0]
                len_l, len_f = leader_meta["length"], follower_meta["length"]
                min_gap = (np.sqrt(np.sum(np.square(pos_leader_xy-pos_follower_xy)))
                        - (len_l + len_f) / 2)
                if min_gap < min_gap_treshold:
                    break
            if np.isclose(crossing_times[follower_id], 0):
                break
            if not next_chunk["speed"].min() < speed_threshold:
                break
            if np.any(condition):
                break
            yield (leader, follower)
            break
import matplotlib.pyplot as plt
%matplotlib auto
# straight_cars = []
# turning_cars = []
# for track_id, chunk in data.groupby(by=["trackId"]):
#     if track_id == 88:
#         print("debug")
#     if car_goes_straight(lane_data, chunk):
#         straight_cars.append(track_id)
#     else:
#         turning_cars.append(track_id)
# print(len(straight_cars))
# print(len(turning_cars))
# pairs = []
# for leader, follower in following_cars(data, lane_data, meta_data, True):
#     pairs.append((leader, follower))
# print(len(pairs))

Using matplotlib backend: <object object at 0x00000266036CE980>


In [4]:
def get_following_cars2(data: pd.DataFrame, lane_data: pd.DataFrame,
                   meta_data: pd.DataFrame, use_xy=False, lanes: list = None,
                   classes : list = ["Car", "Van"]):
    if not lanes:
        lanes = np.unique(lane_data["trackId"])
    times = np.sort(data["time"].unique())[:2]
    fps = np.rint(1 / (times[1] - times[0])).astype(int)
    track_ids = []
    trajectories = data.copy()
    for track_id, chunk in data.groupby(by=["trackId"]):
        lane_counter = Counter(chunk["lane"])
        dominant_lane = max(chunk["lane"], key=lane_counter.get)
        trajectories.loc[chunk.index, "dominantLane"] = dominant_lane
        if dominant_lane not in lanes:
            continue
        straight = car_goes_straight(lane_data, chunk)
        if straight:
            track_ids.append(track_id)
    trajectories = trajectories[trajectories["trackId"].isin(track_ids)]
    # get times as which most cars stop at the intersection
    redlight_peaks = intersection_peaks(trajectories, lane_data, lanes)
    condition = (
        (trajectories["time"].isin(redlight_peaks))
        & (trajectories["lane"].isin(lanes))
    )
    stop_frames = np.sort(trajectories[condition]["frame"].unique())
    frames_to_investigate = []
    for begin, stop in zip(stop_frames[:-1], stop_frames[1:]):
        # capture order every 10 seconds
        ten_secs = 10 * fps
        step_size = np.rint((stop - begin) / ten_secs)
        ten_secs // step_size
        frames_to_investigate.extend(list(np.arange(begin, stop, ten_secs)))
    condition = (trajectories["frame"].isin(frames_to_investigate))
    redlight_situation = trajectories[condition]
    # identifiy pairs of leader follower at stopping positions
    pairs = pd.DataFrame()
    leaders = []
    for group_name, stopped_chunk in redlight_situation.groupby(by=["frame", "lane"]):
        sequence = stopped_chunk.sort_values(by="distanceIntersectionCrossing", ascending=False)
        sequence = sequence[~sequence["trackId"].isin(leaders)]
        # sequence = sequence[sequence["dominantLane"]==group_name[1]]
        sequence["follower"] = np.roll(sequence["trackId"].values, -1)
        # sequence["order"] = np.arange((len(sequence)))
        sequence = sequence.iloc[:-1]
        leaders.extend(list(sequence["trackId"].values))
        pairs = pd.concat((pairs, sequence))
    situations = []
    for index, row in pairs.iterrows():
        situation = np.argmin(redlight_peaks - row["time"])
        situations.append(situation)
    pairs.reset_index(inplace=True)
    pairs["siutation"] = situations
    pairs = pairs.rename(columns={"trackId": "leader"})
    # TODO:criterea
    pairs = pairs[pairs["class"].isin(classes)]
    return pairs[["leader", "follower"]].values
    
    
        
    
def intersection_peaks(trajectories, lane_data, lanes=None):
    # find peaks of when cars stop on lanes at the intersection
    # downscale time
    times = trajectories["time"].unique()
    times = np.sort(times)
    downscale_number = 100
    step_size = times.shape[0] // downscale_number
    times = times[::step_size]

    cars_stopped = np.zeros_like(times)
    for idx, time in enumerate(times):
        if lanes:
            selected_lanes = lanes
        else:
            selected_lanes = np.unique(lane_data["trackId"])
        conditions = (
            (trajectories["time"]==time)
            & (trajectories["speed"]==0)
            & (trajectories["lane"].isin(selected_lanes))
        )
        selected_trajectories = trajectories[conditions]
        cars_stopped[idx] = selected_trajectories["trackId"].unique().shape[0]
    time_step = np.mean(times[1:]-times[:-1])
    time_dist = np.rint(30 / (time_step))
    idxs = find_peaks(cars_stopped, distance=time_dist)[0]
    return times[idxs]

    
selection = get_following_cars2(data, lane_data, meta_data, True, [1, 2])

        

In [5]:
def _get_starting_time(pos_car):
    starting_indexes = np.argwhere(pos_car["speed"].values==0)
    if starting_indexes.shape[0] != 0:
        starting_index = np.clip(starting_indexes[-1,0] + 1, 0,
                                 starting_indexes.shape[0]-1)
    else:
        starting_index = np.argmin(pos_car["speed"]).astype(int)
        # TODO: get starting index by min speed BEFORE intersection
        # crossing
        return pos_car["time"].values[starting_index]
    starting_time = pos_car["time"].values[starting_index]
    return starting_time

def _estimate_parameters(identification: tuple,
                              data_chunk: pd.DataFrame,
                              meta_data: pd.DataFrame):
    leader_chunk = data_chunk[data_chunk["trackId"]==identification[0]]
    follower_chunk = data_chunk[data_chunk["trackId"]==identification[1]]
    starting_time_l = _get_starting_time(leader_chunk)
    leader_start = leader_chunk[leader_chunk["time"]>starting_time_l]
    leader_start.reset_index(drop=True, inplace=True)
    # leader_time = leader_start["time"].values
    local_maxima_i = argrelextrema(
        leader_start["acc"].values, np.greater)[0]
    # local_maxima = leader_time[local_maxima_i]
    if local_maxima_i.shape[0] == 0:
        taccmax_idx_l = np.argmax(leader_start["acc"].values)
    else:
        taccmax_idx_l = local_maxima_i[0]
    # leader_t_acc = leader_start[["time", "acc"]].values[:taccmax_idx_l+1]
    taccmax_l = leader_start["acc"].values[taccmax_idx_l]
    taccmax_l += 0.8 # see EIDM paper by Salles D.
    # TODO: hardcoded speed limit 50km/h
    speed_factor_leader =  leader_chunk["speed"].max() / 13.8889
    if isinstance(identification[1], str):
        if identification == "":
            return None, taccmax_l, None, None, speed_factor_leader, None
    else:
        if np.isnan(identification[1]) or identification[1] is None:
            return None, taccmax_l, None, None, speed_factor_leader, None
    pos_follower = follower_chunk[follower_chunk["time"]==starting_time_l]
    pos_leader = leader_chunk[leader_chunk["time"]==starting_time_l]
    leader_meta, follower_meta = _get_vehicle_meta_data(
        meta_data, identification)
    len_l, len_f = leader_meta["length"], follower_meta["length"]
    # len_l, len_f = pos_leader["length"].values[0], pos_follower["length"].values[0]
    pos_follower_xy = pos_follower[["xCenter", "yCenter"]].values[0]
    pos_leader_xy = pos_leader[["xCenter", "yCenter"]].values[0]
    min_gap = (np.sqrt(np.sum(np.square(pos_leader_xy-pos_follower_xy)))
               - (len_l + len_f) / 2) + 0.1
    starting_time_f = _get_starting_time(follower_chunk)
    startup_delay = np.clip((starting_time_l - starting_time_f - 0.25),
                            0, 2)
    conditions = (
        (follower_chunk["time"].values<=starting_time_f)
        & (np.isclose(follower_chunk["acc"].values,0))
    )
    if not any(conditions):
        # L134 F137
        conditions = (
            (follower_chunk["time"].values<=starting_time_l)
            & (np.isclose(follower_chunk["acc"].values, 0, atol=0.1))
        )
        if not any(conditions):
            conditions = np.zeros_like(follower_chunk["acc"].values)
            conditions[0] = 1
            conditions = conditions.astype(bool)
    starting_time_f = follower_chunk[conditions]["time"].values[-1]
    follower_start = follower_chunk[
        follower_chunk["time"]>=starting_time_f]
    speed_factor_follower =  follower_chunk["speed"].max() / 13.8889
    follower_start.reset_index(drop=True, inplace=True)
    follower_time = follower_start["time"].values
    local_maxima_i = argrelextrema(
        follower_start["acc"].values, np.greater)[0]
    # local_maxima = follower_time[local_maxima_i]
    if local_maxima_i.shape[0] == 0:
        taccmax_idx_f = np.argmax(follower_start["acc"].values)
    else:
        taccmax_idx_f = next(
            x for x in local_maxima_i if follower_start["acc"].values[x] > 1)
    taccmax_f = follower_start["acc"].values[taccmax_idx_f]
    # taccmax_f += 0.8 # see EIDM paper by Salles D. # not needed with the new
    # files
    taccmax_dur_f = follower_time[taccmax_idx_f] - follower_time[0]
    follower_t_acc = follower_start[["time", "acc"]].values[:taccmax_idx_f+1]
    curve = _acceleration_curve_factory(taccmax_dur_f, starting_time_f)
    solutions = ()
    try:
        solutions = op.curve_fit(
            curve,
            follower_t_acc[:,0],
            follower_t_acc[:,1] / follower_t_acc[-1,1])
    except RuntimeError:
        # could not estimate parameters
        pass
    m_beg, m_flat = None, None
    for solution in solutions:
        if np.all(solution != np.inf):
            m_beg, m_flat = solution
            break
    import matplotlib.pyplot as plt
    plt.figure()
    plt.plot(follower_t_acc[:,0], follower_t_acc[:,1] / follower_t_acc[-1,1])
    plt.plot(follower_t_acc[:,0], curve(follower_t_acc[:,0], m_beg, m_flat))
    plt.show()
    params = (min_gap, taccmax_f, m_beg, m_flat, speed_factor_follower,
              startup_delay)
    return params
for identification in selection:
    identification = (identification[0], identification[1], 1)
    _estimate_parameters(identification, data, meta_data)

KeyboardInterrupt: 

: 